In [ ]:
import random

server_list

def getRandomString():
    
    
def startSentinel():

    
def killSentinel():

    

#start redis + DT server + notify sentinel
#update server_list
def startServers(numServers):
    

#kill the redis server at addr
#update server_list
def killServer(addr):
    

#get and write the status to some log
def getStatus():
    
#compute num migrations from two succesie status calls and write to file
def getNumMigrations():